In [2]:
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sage_data_client import query

In [3]:
def get_plugins_per_node(jobs):
    for _, j in jobs.iterrows():
        if isinstance(j["science_goal"], dict):
            # print(j["science_goal"]["sub_goals"])
            sub_goals = j["science_goal"]["sub_goals"]
            for sub_goal in sub_goals:
                for plugin in sub_goal["plugins"]:
                    yield pd.Series(data=[j["job_id"], sub_goal["name"], plugin["name"], os.path.basename(plugin["plugin_spec"]["image"])], index=["job", "vsn", "plugin_name", "image"])
    

In [4]:
url = "https://es.sagecontinuum.org/api/v1/jobs/list"
df = pd.read_json(url).T

In [5]:
df

,name,job_id,user,email,notification_on,plugins,node_tags,nodes,science_rules,success_criteria,science_goal,state
1,neon-v001,1,yonghokim,,None,"[{'name': 'imagesampler-neon40', 'plugin_spec'...",None,{'V001': None},[imagesampler-neon40: cronjob('imagesampler-ne...,[WallClock(1d)],"{'id': 'e16d430c-d7e5-4d56-5b36-67fd18f4e829',...","{'last_state': 'Running', 'last_updated': '202..."
10,sage-uoregon-job,10,yonghokim,,None,"[{'name': 'imagesampler-top', 'plugin_spec': {...",None,{'W019': None},"[imagesampler-top: cronjob('imagesampler-top',...",[WallClock(1d)],"{'id': '95038712-e4f8-4caa-4a2d-cbcaa9f2d28c',...","{'last_state': 'Running', 'last_updated': '202..."
11,sage-urban-job,11,yonghokim,,None,"[{'name': 'imagesampler-top', 'plugin_spec': {...","[WSN, SAGE, Urban, microphone]","{'W015': 1, 'W024': 1, 'W026': 1, 'W027': 1, '...","[imagesampler-top: cronjob('imagesampler-top',...",[WallClock(1d)],"{'id': '8574933f-3e2f-419e-5776-1b2fcbeaa6fe',...","{'last_state': 'Running', 'last_updated': '202..."
12,sage-utah-job,12,yonghokim,,None,"[{'name': 'imagesampler-top', 'plugin_spec': {...",None,{'W029': None},"[imagesampler-top: cronjob('imagesampler-top',...",[WallClock(1d)],"{'id': '5acb69aa-fc5a-4076-6f42-415e60d1a779',...","{'last_state': 'Running', 'last_updated': '202..."
13,sage_w014_image_sampler,13,yonghokim,,None,"[{'name': 'imagesampler-top', 'plugin_spec': {...",None,{'W014': None},"[imagesampler-top: cronjob('imagesampler-top',...",None,"{'id': '86a8fad0-d86d-4fc1-4675-09a20cf08182',...","{'last_state': 'Running', 'last_updated': '202..."
14,sage_w028,14,yonghokim,,None,"[{'name': 'imagesampler-top', 'plugin_spec': {...",None,{'W028': None},"[imagesampler-top: cronjob('imagesampler-top',...",None,"{'id': 'bb2b9542-df72-4426-7ab8-df9e05004783',...","{'last_state': 'Running', 'last_updated': '202..."
15,sage_w06d_image_sampler,15,yonghokim,,None,"[{'name': 'imagesampler-top', 'plugin_spec': {...",None,{'W06D': None},"[imagesampler-top: cronjob('imagesampler-top',...",None,"{'id': 'e676ab6a-527d-43ee-6099-8fe11f687639',...","{'last_state': 'Running', 'last_updated': '202..."
16,vto-video-sampler,16,yonghokim,,None,"[{'name': 'video-sampler-left', 'plugin_spec':...",None,{'W023': None},[video-sampler-left: cronjob('video-sampler-le...,[WallClock(1d)],"{'id': 'a7543196-553e-4703-4066-c01675d6113c',...","{'last_state': 'Running', 'last_updated': '202..."
17,test-pipeline,17,seanshahkarami,,None,"[{'name': 'test-pipeline', 'plugin_spec': {'im...",None,{'W030': None},"[image-sampler: cronjob('test-pipeline', '* * ...",None,NaN,"{'last_state': 'Removed', 'last_updated': '202..."
18,vto-job,18,yonghokim,,None,"[{'name': 'imagesampler-left', 'plugin_spec': ...","[WSN, VTO]","{'W01E': 1, 'W02D': 1, 'W02E': 1, 'W02F': 1, '...",[imagesampler-left: cronjob('imagesampler-left...,[WallClock(1d)],"{'id': '07763131-652f-476e-7bcf-38b2cfc8e6b3',...","{'last_state': 'Running', 'last_updated': '202..."


In [6]:
data = pd.DataFrame(get_plugins_per_node(df))

In [7]:
grouped_by_image = data.groupby(by=["image"])

In [8]:
data

,job,vsn,plugin_name,image
0,1,V001,imagesampler-neon40,imagesampler:0.3.0
1,1,V001,imagesampler-neon41,imagesampler:0.3.0
2,1,V001,object-counter-neon40,object-counter:0.5.1
3,1,V001,object-counter-neon41,object-counter:0.5.1
4,1,V001,smoke-detector-neon41,wildfire-smoke-detection:0.5.0
...,...,...,...,...
294,9,W02B,imagesampler-bottom,imagesampler:0.3.0
295,9,W02B,cloud-cover-top,cloud-cover:0.1.3
296,9,W02B,cloud-motion-top,cloud-motion:0.22.2.27
297,9,W02B,object-counter-bottom,object-counter:0.5.1


In [9]:
df2 = data.drop_duplicates(subset=["image", "vsn"])

Count number of Waggle nodes that run the edge plugins

In [10]:
df2.groupby("image")["vsn"].count()

image
avian-diversity-monitoring:0.2.4     3
cloud-cover:0.1.3                   32
cloud-motion:0.22.2.27              26
gpu-stress-test:latest               1
imagesampler:0.3.0                  58
object-counter:0.5.1                55
sound-event-detection:0.1.1         27
surface-water-classifier:0.0.5       1
video-sampler:0.2.4                  1
water-depth-estimator:0.0.1          3
wildfire-smoke-detection:0.5.0       7
Name: vsn, dtype: int64

In [11]:
df2.groupby("vsn")["image"].count()

vsn
V001    4
V002    4
V003    5
V004    5
V005    4
V006    5
W014    1
W015    5
W019    5
W01E    2
W021    5
W023    2
W024    6
W026    5
W027    5
W028    4
W029    6
W02B    5
W02C    5
W02D    2
W02E    2
W02F    2
W040    2
W045    5
W046    2
W047    2
W048    2
W049    2
W04A    2
W05F    2
W060    2
W061    2
W062    2
W063    2
W064    2
W069    5
W06A    5
W06D    1
W06F    5
W078    5
W079    5
W07A    5
W07B    5
W07C    5
W07D    5
W07E    5
W07F    5
W080    5
W081    5
W084    6
W087    5
W088    5
W089    2
W08A    2
W08F    2
W090    2
W091    2
W092    2
W093    2
Name: image, dtype: int64